In [8]:
import pandas as pd
import json
import ipywidgets as widgets
from IPython import display

NUMBER_OF_NODES = 10

lines = sc.textFile('file:///home/duan/github/customer_segmentation/data/journey_log/msnbc.numbered')
lines_nonempty = lines.filter( lambda x: len(x) > 0 )

cust_df = pd.DataFrame.from_csv('/home/duan/github/customer_segmentation/data/cust_profile_df/cust_sentiment_gender_df.csv')
global_selected_cust_df = cust_df
cust_journey_df=pd.DataFrame.from_csv('/home/duan/github/customer_segmentation/data/cust_profile_df/cust_journey_df.csv.new.1')


In [9]:
# Write the journey df to disk for later on computation.
def build_journey_all(lines_nonempty):
    wordcounts = build_journey_with_customer_filter(lines_nonempty, cust_list = None)
    from pyspark.sql import SQLContext, Row
    sqlContext = SQLContext(sc)
    # Load a text file and convert each line to a Row.  
    journey_rows = wordcounts.map(lambda p: Row(visitor_count=p[0], journey= p[1][0], visitor_list=p[1][1]))
    # Infer the schema, and register the DataFrame as a table.
    schemaPeople = sqlContext.createDataFrame(journey_rows)
    # schemaPeople.registerTempTable("journey_table")
    cust_journey_df = schemaPeople.toPandas()
    cust_journey_df.to_csv('/home/duan/github/customer_segmentation/data/cust_profile_df/cust_journey_df.csv.new.1')
    return cust_journey_df
# Only for first time, or schedule it every night.
# cust_journey_df = build_journey_all(lines_nonempty)

In [10]:
from nvd3piewidget import (jouney_covered_check, 
                           collect_customer_from_journey, 
                           build_journey_with_customer_filter, 
                           selected_cust_sankey_json,
                           selected_cust_sankey_json
                           )
from nvd3piewidget import D3SankeyWidget,NVD3PieWidget,NVD3BarWidget


In [11]:
def update_bar_chart(global_selected_cust_df):
    hist1 = global_selected_cust_df['sentiment'].value_counts()
    hist1.sort_index(ascending=True,inplace=True)
    hist_dict=hist1.to_dict()
    bar_value = [{"y": int(hist_dict[k]), "x": str(int(k))}  for k in hist_dict.keys() ]
    data_barchart=[{"values": bar_value, 
                    "key": "Sentiment Historgram", 
                    "yAxis": "1"}]
    bar_chart.json_data = json.dumps(data_barchart)
def update_pie_chart(global_selected_cust_df):
    hist1 = global_selected_cust_df['gender'].value_counts()
    # hist1.sort_index(ascending=True,inplace=True)
    hist_dict=hist1.to_dict()
    gender_dict = {'F':'Female', 'M':'Male'}
    bar_value = [{"value": int(hist_dict[k]), "label": gender_dict[k]}  
                 for k in hist_dict.keys() ]
    data_piechart=[{"values": bar_value, 
                    "key": "Sentiment Historgram" }]
    pie_chart.json_data = json.dumps(data_piechart)

def update_sankey_chart(global_selected_cust_df):
    new_json_string = selected_cust_sankey_json(lines_nonempty, global_selected_cust_df[['cust_id']] ,NUMBER_OF_NODES, sc)
    sankey_journey_widget.node_link_json = new_json_string
# sankey_journey_widget.node_link_json = json_string
# update_sankey_chart(global_selected_cust_df[global_selected_cust_df['gender']  == 'M' ] )

def update_all_chart(global_selected_cust_df):
    update_bar_chart(global_selected_cust_df)
    update_pie_chart(global_selected_cust_df)
    update_sankey_chart(global_selected_cust_df)
    cust_count_string = '# of Customers: ' + str(global_selected_cust_df['cust_id'].count() )
    comment_field.value = cust_count_string


In [12]:
from IPython.display import HTML
HTML('''
<style>
#allchart {
   height: 350px;
   overflow: hidden;
}
#barchart {
    width: 420px;
    height: 350px;
    float: right;
}
#piechart {
    float: none; /* not needed, just for clarification */
    height: 350px;
    width: 420px;
    overflow: hidden;
</style>

<div id="allchart" >
    <div id="barchart"  >  &nbsp; &nbsp;  Sentiment Distribution
        <svg ></svg>
    </div>
    <div id="piechart"  > &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  Gender Distribution
        <svg ></svg>
    </div>
</div>

''')

In [13]:
json_string = selected_cust_sankey_json(lines_nonempty, None)
sankey_journey_widget = D3SankeyWidget(node_link_json = json_string, height = 300, width = 960 )
display.display(sankey_journey_widget)

In [14]:
data_barchart=[{'key': 'Sentiment Historgram',
  'values': [    {'x': '1', 'y':  75 },
   {'x': '2', 'y':  407 }, 
   {'x': '5', 'y':  204 }, 
   {'x': '7', 'y':  15 },
   {'x': '8', 'y':  1 }],'yAxis': '1'}]

bar_chart = NVD3BarWidget(json_data=json.dumps(data_barchart))
# print('         Sentiment Distribution')
display.display(bar_chart)
update_bar_chart(global_selected_cust_df)

In [15]:
data_piechart=[{'key': 'Sentiment Historgram', 
                'values': [{'label': 'F', 'value': 2007},{'label': 'M', 'value': 107}]}];

pie_chart = NVD3PieWidget(value=5, json_data=json.dumps(data_piechart))
# print('         Gender Distribution')
display.display(pie_chart)
update_pie_chart(global_selected_cust_df)

In [16]:
'''
global_selected_cust_df = cust_df[cust_df['gender']  == 'F' ]

update_bar_chart(global_selected_cust_df)
update_pie_chart(global_selected_cust_df)
update_sankey_chart(global_selected_cust_df)
'''

"\nglobal_selected_cust_df = cust_df[cust_df['gender']  == 'F' ]\n\nupdate_bar_chart(global_selected_cust_df)\nupdate_pie_chart(global_selected_cust_df)\nupdate_sankey_chart(global_selected_cust_df)\n"

In [17]:
# Buttons: Reset, Select


filter_by_journey_button = widgets.Button(description="Filter by Journey")
filter_by_journey_button.button_style = 'info'
filter_by_journey_button.margin = 5
filter_by_journey_button.height = 3

display.display(filter_by_journey_button)



def on_filter_by_journey_button_clicked(b):
    global global_selected_cust_df
    # print(d3sankeyExample.selected_json)
    if len(sankey_journey_widget.selected_json) < 3 :
        print('please select at least one node.')
        return
    ss_json = json.loads(sankey_journey_widget.selected_json)
    new_ss = {k: v for k, v in ss_json.items() if v == "y"}
    selected_journey=new_ss.keys()
    selected_journey_df=cust_journey_df[cust_journey_df.apply(lambda x: jouney_covered_check(selected_journey,x['journey']) , axis=1)]
    collecteddf = {'cust_id':  collect_customer_from_journey(selected_journey_df)}
    df_new = pd.DataFrame(collecteddf, columns = ['cust_id' ])
    df_new = df_new.apply(pd.to_numeric, errors='ignore').sort_values(by='cust_id')
    collected_df = pd.merge(df_new, cust_df, how='inner', left_on='cust_id', right_on='cust_id')
    global_selected_cust_df = collected_df
    
    update_all_chart(global_selected_cust_df) 

filter_by_journey_button.on_click(on_filter_by_journey_button_clicked)

In [18]:
# global_selected_cust_df=cust_df

In [19]:

filter_by_gender_button = widgets.Button(description="Filter by Gender")
filter_by_gender_button.button_style = 'info'
filter_by_gender_button.margin = 5
filter_by_gender_button.height = 3

display.display(filter_by_gender_button)



def on_filter_by_gender_button_clicked(b):
    global global_selected_cust_df
    if len(pie_chart.selected_json) < 3 :
        print('please select at least one gender.')
        return
    ss_json = json.loads(pie_chart.selected_json)
    new_ss = {k: v for k, v in ss_json.items() if v == "y"}
    reverse_gender_dict = {'Female':'F', 'Male':'M'}
    selected_gender=reverse_gender_dict [ list(new_ss.keys())[0] ]
    #print('gender=' + selected_gender)

    global_selected_cust_df = global_selected_cust_df[global_selected_cust_df['gender']== selected_gender] # == 'F'
    
    if(global_selected_cust_df['cust_id'].count() < 1):
        print('found no match, do not update charts')
        return
    # print(global_selected_cust_df.count()))
    update_all_chart(global_selected_cust_df) 

filter_by_gender_button.on_click(on_filter_by_gender_button_clicked)

In [20]:

filter_reset_button = widgets.Button(description="Reset Filters")
filter_reset_button.button_style = 'info'
filter_reset_button.margin = 5
filter_reset_button.height = 3

display.display(filter_reset_button)



def on_filter_reset_button_clicked(b):
    global global_selected_cust_df
    global cust_df
    
    global_selected_cust_df = cust_df
    update_all_chart(global_selected_cust_df) 

filter_reset_button.on_click(on_filter_reset_button_clicked)

In [21]:
'''
ss_json = json.loads(pie_chart.selected_json)
new_ss = {k: v for k, v in ss_json.items() if v == "y"}
selected_gender=list(new_ss.keys())[0]
global_selected_cust_df = global_selected_cust_df[global_selected_cust_df['gender'] 
                                                  == selected_gender] # == 'F'
update_bar_chart(global_selected_cust_df)
update_pie_chart(global_selected_cust_df)
update_sankey_chart(global_selected_cust_df)

'''

'\nss_json = json.loads(pie_chart.selected_json)\nnew_ss = {k: v for k, v in ss_json.items() if v == "y"}\nselected_gender=list(new_ss.keys())[0]\nglobal_selected_cust_df = global_selected_cust_df[global_selected_cust_df[\'gender\'] \n                                                  == selected_gender] # == \'F\'\nupdate_bar_chart(global_selected_cust_df)\nupdate_pie_chart(global_selected_cust_df)\nupdate_sankey_chart(global_selected_cust_df)\n\n'

In [22]:
grade_top_row = widgets.HBox()

hid_txt = widgets.HTML("Segment Name:")
hid_txt.font = 'helvetica'
hid_txt.font_size = 30
hid_txt.margin = 5
algo_id_field = widgets.Text()
algo_id_field.height = 35
algo_id_field.value='Segment_1'

grade_txt = widgets.HTML("Segment Type:")
grade_select = widgets.Dropdown(options=["Marketing", "Promotion", "Retension"])



grade_btn = widgets.Button(description="Create Segment")
grade_btn.margin = 5
grade_btn.button_style = 'info'


grade_top_row.children = (hid_txt, algo_id_field, grade_txt, grade_select, grade_btn)
grade_top_row.align = 'center'

comment_field = widgets.Textarea(value='')
comment_field.margin = 5
comment_field.width = 378
comment_field.cols = 55

grade_form = widgets.VBox()
grade_form.children = (grade_top_row, comment_field )


def on_create_button_clicked(b):
    # print(d3sankeyExample.selected_json)
    if global_selected_cust_df['cust_id'].count() < 2 :
        print('Please select at least one customer.')
        return
    cust_list1 = global_selected_cust_df['cust_id'].tolist()
    cust_list = [str(int(x)) for x in cust_list1]
    # print(cust_list)
    new_seg = {'Seg_name': algo_id_field.value,
               'Seg_type': grade_select.value,
               'cust_ids': ','.join(cust_list)
              }
    display.clear_output(wait=True) 
    comment_field.value = json.dumps(new_seg)
    

display.display(grade_top_row)
grade_btn.on_click(on_create_button_clicked)


In [23]:
display.display(comment_field)